In [1]:
!python -m pip install mysql-connector

In [2]:
import pandas as pd
import mysql.connector

In [3]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [4]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="forex"
)

In [5]:
import os

def ask_user(msg):
    print(msg)
    response = ''
    while response.lower() not in {"yes", "no"}:
        response = input("Please enter yes or no: ")
    return response.lower() == "yes"

def cleanup_file(input_file_name, output_file_name):
    head_sample = 'Timestamp,Bid price\n'
    row_sample = '2012-01-01 22:00:36.416,1.29368\n'

    with open(input_file_name) as fin, open(output_file_name, 'a') as fout:

        for index, line in enumerate(fin):
            if index==0:
                fout.write(line)
                continue

            if len(line) < len(row_sample):
                characters_needed = len(row_sample) - len(line)
                line = line[:-1] + '0'*characters_needed + '\n'

            fout.write(line)

input_file_name = 'TickStory/EURUSD.csv'
output_file_name = 'TickStory/clean/EURUSD.csv'

if ask_user("Do you want to cleanup EURUSD.csv ?"):
    cleanup_file(input_file_name, output_file_name)

Do you want to cleanup EURUSD.csv ?
Please enter yes or no: no


In [6]:
# file_name = 'TickStory/clean/EURUSD.csv'

# import time
# start = time.time()

# with open(file_name) as f:
#     total_line_count = sum(1 for line in f)
    
# print(total_line_count, "lines")

# end = time.time()
# print("Time Taken: ", end - start)

In [7]:
import os

file_name = 'TickStory/clean/EURUSD.csv'

with open(file_name) as f:
    head = f.readline()
    row = f.readline()

file_size = os.path.getsize(file_name)

dividant = file_size - len(head)-1 #one negated for /r in the first line
divider = len(row)+1 #one added for /r in each line

total_line_count = dividant//divider
print("total rows count: ", total_line_count)

total rows count:  194760745


In [8]:
with open("semaphore.txt") as semaphore:
    write_count = semaphore.read()
    write_count = 0 if write_count == '' else int(write_count)
    print(write_count)

19477


In [9]:
def load_ticks_to_mysql():
    chunksize = 10000
    eurusd_ticks = pd.read_csv(file_name, chunksize=chunksize)

    with open("semaphore.txt") as semaphore:
        write_count = semaphore.read()
        write_count = 0 if write_count == '' else int(write_count)

    for index, tick in enumerate(eurusd_ticks):
        timestamps = tick['Timestamp'].tolist()
        prices = tick['Bid price'].tolist()

        if index >= write_count:
            sql = "INSERT INTO price (price, timestamp) VALUES (%s, %s)"
            values = []

            for (timestamp, price) in zip(timestamps, prices):
                values.append( (round(price, 5), timestamp) )

            mycursor = db.cursor(prepared=True)
            mycursor.executemany(sql, values)
            db.commit()
            mycursor.close()

            with open("semaphore.txt",  "w+") as semaphore:
                semaphore.write(str(index+1))

        suffix = "(" + str(index*chunksize) + "/" + str(total_line_count) + ")"
        printProgressBar(index*chunksize, total_line_count, "progress: ", suffix)
        
if ask_user("Do you want to load EURUSD.csv to the mysql database?"):
    load_ticks_to_mysql()


Do you want to load EURUSD.csv to the mysql database?
Please enter yes or no: no


**SQL Query:**
```
(SELECT * FROM `price` AS p WHERE p.timestamp>='2012-01-11 08:52:46' AND p.price >= (1.2773+0.0050) ORDER BY timestamp ASC LIMIT 1)
UNION
(SELECT * FROM `price` AS p WHERE p.timestamp>='2012-01-11 08:52:46' AND p.price <= (1.2773-0.0050) ORDER BY timestamp ASC LIMIT 1)
```